In [1]:

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import glob
import os

In [2]:
# df = pd.read_csv(r"C:\Users\nandi\Downloads\DOB_Permit_data-20251116T004852Z-1-001\DOB_Permit_data\data_9.csv")
all_files = sorted(glob.glob('data/kmeans_clustered/distributed_kmeans_chunk_*.csv'))
df = pd.concat([pd.read_csv(f) for f in all_files], ignore_index=True)

In [3]:
df.columns

Index(['LATITUDE', 'LONGITUDE', 'COUNCIL_DISTRICT', 'CENSUS_TRACT',
       'Filing Date', 'Issuance Date', 'Bldg Type', 'Zip Code', 'Filing_Year',
       'Filing_Month', 'Filing_Quarter', 'Permit_Age_Days', 'Permit Type_DM',
       'Permit Type_EQ', 'Permit Type_EW', 'Permit Type_FO', 'Permit Type_NB',
       'Permit Type_PL', 'Permit Type_SG', 'Job Type_A2', 'Job Type_A3',
       'Job Type_DM', 'Job Type_NB', 'Job Type_SG', 'Permit Status_ISSUED',
       'Permit Status_RE-ISSUED', 'Permit Status_REVOKED',
       'Permit Status_Unknown', 'Residential_YES', 'BOROUGH_BROOKLYN',
       'BOROUGH_MANHATTAN', 'BOROUGH_QUEENS', 'BOROUGH_STATEN ISLAND',
       'Work Type_CC', 'Work Type_EQ', 'Work Type_FA', 'Work Type_FB',
       'Work Type_FP', 'Work Type_FS', 'Work Type_MH', 'Work Type_OT',
       'Work Type_PL', 'Work Type_SD', 'Work Type_SP', 'Work Type_Unknown',
       'Permit Subtype_CH', 'Permit Subtype_EA', 'Permit Subtype_FA',
       'Permit Subtype_FB', 'Permit Subtype_FN', 'Permit S

In [20]:
df.head()

,Unnamed: 0,BOROUGH,Bin #,House #,Street Name,Job #,Job doc. #,Job Type,Self_Cert,Block,...,Owner’s House State,Owner’s House Zip Code,Owner's Phone #,DOBRunDate,PERMIT_SI_NO,LATITUDE,LONGITUDE,COUNCIL_DISTRICT,CENSUS_TRACT,NTA_NAME
0,3584933,QUEENS,4203062,107-36,108 STREET,440530472,1,A2,N,9545.0,...,NaN,NaN,3473572166,05/09/2022 00:00:00,770543,40.681301,-73.832621,32.0,96.0,South Ozone Park
1,3584934,MANHATTAN,1048692,201,EAST 83 STREET,104202907,1,A2,Y,1529.0,...,NaN,NaN,2122474975,05/09/2022 00:00:00,770544,40.776734,-73.955066,5.0,138.0,Yorkville
2,3584935,MANHATTAN,1059562,439,WEST 126 STREET,101153895,1,A1,NaN,1967.0,...,NaN,NaN,2128702138,05/09/2022 00:00:00,770545,40.813025,-73.954724,7.0,21303.0,Manhattanville
3,3584936,QUEENS,4038278,78-04,45 AVENUE,402297732,1,A2,Y,1530.0,...,NaN,NaN,7186397388,05/09/2022 00:00:00,770546,40.741189,-73.886482,25.0,485.0,Elmhurst-Maspeth
4,3584937,QUEENS,4127187,186-05,UNDERHILL AVENUE,401240509,1,A1,NaN,5622.0,...,NaN,NaN,7183577850,05/09/2022 00:00:00,770547,40.746435,-73.791192,20.0,1417.0,Auburndale


In [6]:
hotspot_cols = ['LATITUDE', 'LONGITUDE', 'COUNCIL_DISTRICT', 'CENSUS_TRACT',
      'Bldg Type', 'Filing_Year',
       'Filing_Month', 'Filing_Quarter', 'Permit_Age_Days', 'Permit Type_DM',
       'Permit Type_EQ', 'Permit Type_EW', 'Permit Type_FO', 'Permit Type_NB',
       'Permit Type_PL', 'Permit Type_SG', 'Job Type_A2', 'Job Type_A3',
       'Job Type_DM', 'Job Type_NB', 'Job Type_SG', 'Permit Status_ISSUED',
       'Permit Status_RE-ISSUED', 'Residential_YES']

In [7]:
df_hotspot = df[hotspot_cols].copy()

In [8]:
df_hotspot = df_hotspot.dropna(subset=['LATITUDE', 'LONGITUDE'])

In [9]:
# # Datetime conversion for date fields
# for date_col in ['Filing Date', 'Issuance Date']:
#     if date_col in df_hotspot.columns:
#         df_hotspot[date_col] = pd.to_datetime(df_hotspot[date_col], errors='coerce')

In [10]:
# Create derived temporal features
# if 'Filing Date' in df_hotspot.columns:
#     df_hotspot['Filing_Year'] = df_hotspot['Filing Date'].dt.year
#     df_hotspot['Filing_Month'] = df_hotspot['Filing Date'].dt.month
#     df_hotspot['Filing_Quarter'] = df_hotspot['Filing Date'].dt.quarter

# if set(['Filing Date', 'Issuance Date']).issubset(df_hotspot.columns):
#     df_hotspot['Permit_Age_Days'] = (df_hotspot['Issuance Date'] - df_hotspot['Filing Date']).dt.days


In [11]:
# df_hotspot['Residential'].fillna("NO", inplace=True)

In [12]:
# # Categorical NA fill and cardinality reduction
# for cat_col in ['Permit Type', 'Job Type', 'Residential', "Permit Status"]:
#     if cat_col in df_hotspot.columns:
#         df_hotspot[cat_col] = df_hotspot[cat_col].fillna('Unknown')
#         if df_hotspot[cat_col].nunique() > 20:  # High cardinality: take top 19, rest as "Other"
#             top_n = df_hotspot[cat_col].value_counts().nlargest(19).index
#             df_hotspot[cat_col] = df_hotspot[cat_col].where(df_hotspot[cat_col].isin(top_n), 'Other')


In [13]:
# # One-hot encode low-cardinality categoricals
# df_hotspot = pd.get_dummies(df_hotspot, columns=['Permit Type', 'Job Type', 'Permit Status', 'Residential'], drop_first=True)

In [ ]:
# df_hotspot.head()
df_hotspot['LATITUDE_ORIG'] = df_hotspot['LATITUDE'].copy()
df_hotspot['LONGITUDE_ORIG'] = df_hotspot['LONGITUDE'].copy()

In [15]:
# from sklearn.preprocessing import StandardScaler

# scaler = StandardScaler()
# numeric_cols = ['LATITUDE', 'LONGITUDE', 'COUNCIL_DISTRICT', 'CENSUS_TRACT', 'Bldg Type']
# for col in numeric_cols:
#     if col in df_hotspot.columns:
#         coldata = df_hotspot[[col]].values
#         df_hotspot[col] = scaler.fit_transform(coldata)

# # df_hotspot is now clean and ready for clustering
# print(df_hotspot.head())

In [16]:
from sklearn.cluster import DBSCAN

In [17]:
# borough_type_cols = [w for w in df_hotspot.columns if w.startswith('BOROUGH_')]
# work_type_cols = [w for w in df_hotspot.columns if w.startswith('Work Type_')]
# permit_subtype_cols = [w for w in df_hotspot.columns if w.startswith('Permit Subtype_')]

# ---- 1. Define features for clustering ----
features = ['LATITUDE', 'LONGITUDE', 'COUNCIL_DISTRICT', 'CENSUS_TRACT',
      'Bldg Type', 'Filing_Year',
       'Filing_Month', 'Filing_Quarter', 'Permit_Age_Days', 'Permit Type_DM',
       'Permit Type_EQ', 'Permit Type_EW', 'Permit Type_FO', 'Permit Type_NB',
       'Permit Type_PL', 'Permit Type_SG', 'Job Type_A2', 'Job Type_A3',
       'Job Type_DM', 'Job Type_NB', 'Job Type_SG', 'Permit Status_ISSUED',
       'Permit Status_RE-ISSUED', 'Residential_YES']

features_for_clustering = features
features_for_clustering = [w for w in features_for_clustering if not w.endswith('Unknown')]


In [18]:
features_for_clustering

['LATITUDE',
 'LONGITUDE',
 'COUNCIL_DISTRICT',
 'CENSUS_TRACT',
 'Bldg Type',
 'Filing_Year',
 'Filing_Month',
 'Filing_Quarter',
 'Permit_Age_Days',
 'Permit Type_DM',
 'Permit Type_EQ',
 'Permit Type_EW',
 'Permit Type_FO',
 'Permit Type_NB',
 'Permit Type_PL',
 'Permit Type_SG',
 'Job Type_A2',
 'Job Type_A3',
 'Job Type_DM',
 'Job Type_NB',
 'Job Type_SG',
 'Permit Status_ISSUED',
 'Permit Status_RE-ISSUED',
 'Residential_YES']

In [19]:
df_hotspot.dropna(inplace=True)

In [20]:
X = df_hotspot[features_for_clustering].astype(float).values

In [21]:
dbscan = DBSCAN(eps=0.1, min_samples=50, n_jobs=20)
df_hotspot['cluster'] = dbscan.fit_predict(X)

In [25]:
df_hotspot.head()

,LATITUDE,LONGITUDE,COUNCIL_DISTRICT,CENSUS_TRACT,Bldg Type,Filing_Year,Filing_Month,Filing_Quarter,Permit_Age_Days,Permit Type_DM,...,Job Type_A2,Job Type_A3,Job Type_DM,Job Type_NB,Job Type_SG,Permit Status_ISSUED,Permit Status_RE-ISSUED,Residential_YES,cluster,is_hotspot
0,0.246542,-0.475496,-1.155621,-0.300199,0.617060,2022.0,5.0,2.0,0.0,False,...,False,True,False,False,False,True,False,False,-1,False
1,-2.076449,-2.588673,1.793092,0.505196,-1.620588,2022.0,5.0,2.0,0.0,False,...,True,False,False,False,False,True,False,True,-1,False
2,-1.080717,-1.646804,1.672736,-0.299011,-1.620588,2022.0,5.0,2.0,0.0,False,...,True,False,False,False,False,True,False,True,-1,False
3,-2.326198,-2.341327,1.793092,0.438349,-1.620588,2022.0,5.0,2.0,0.0,False,...,True,False,False,False,False,True,False,True,-1,False
4,-2.035246,-1.928620,1.732914,-0.296493,-1.620588,2022.0,5.0,2.0,0.0,False,...,True,False,False,False,False,True,False,True,-1,False


In [ ]:
df_hotspot['is_hotspot'] = df_hotspot['cluster'] != -1

In [23]:
hotspot_summary = (
    df_hotspot.query("is_hotspot")
    .groupby(['cluster', 'Filing_Year', 'Filing_Month'])
    .agg(
        n_permits=('LATITUDE', 'size'),
        long_centroid=('LONGITUDE', 'median'),
        lat_centroid=('LATITUDE', 'median')
        # No NTA columns included
    )
    .reset_index()
)


# If you want to add other categorical summaries, such as most common permit type or job type:
def most_common_permit_type(group):
    permit_type_cols = [col for col in df_hotspot.columns if col.startswith('Permit Type_')]
    # Return the permit type column with maximum sum in the group
    if permit_type_cols:
        sums = {col: group[col].sum() for col in permit_type_cols}
        return max(sums, key=sums.get).replace('Permit Type_', '')
    return 'Unknown'

def most_common_job_type(group):
    job_type_cols = [col for col in df_hotspot.columns if col.startswith('Job Type_')]
    if job_type_cols:
        sums = {col: group[col].sum() for col in job_type_cols}
        return max(sums, key=sums.get).replace('Job Type_', '')
    return 'Unknown'

hotspot_summary['common_permit_type'] = hotspot_summary.apply(
    lambda row: most_common_permit_type(
        df_hotspot[(df_hotspot['cluster'] == row['cluster']) &
                   (df_hotspot['Filing_Year'] == row['Filing_Year']) &
                   (df_hotspot['Filing_Month'] == row['Filing_Month'])]
    ), axis=1
)


hotspot_summary['common_job_type'] = hotspot_summary.apply(
    lambda row: most_common_job_type(
        df_hotspot[(df_hotspot['cluster'] == row['cluster']) &
                   (df_hotspot['Filing_Year'] == row['Filing_Year']) &
                   (df_hotspot['Filing_Month'] == row['Filing_Month'])]
    ), axis=1
)


# Only keep relevant columns for reporting
hotspot_summary = hotspot_summary[
    ['cluster', 'Filing_Year', 'Filing_Month', 'n_permits', 'long_centroid', 'lat_centroid', 'common_permit_type', 'common_job_type']
].sort_values(['Filing_Year', 'Filing_Month', 'n_permits'], ascending=[True, True, False])

print(hotspot_summary.head(10))

      cluster  Filing_Year  Filing_Month  n_permits  long_centroid  \
2064     2064       1990.0           5.0         66      -0.331002   
2092     2092       1990.0           6.0        283      -0.368428   
2091     2091       1990.0           6.0         67      -0.709825   
2128     2128       1990.0           7.0        376      -0.377947   
2129     2129       1990.0           7.0        104      -0.705613   
2133     2133       1990.0           7.0         68      -0.388834   
2169     2169       1990.0           8.0        425      -0.372570   
2170     2170       1990.0           8.0        120      -0.733413   
2172     2172       1990.0           8.0         69      -0.349758   
385       385       1990.0           9.0        376      -0.383273   

      lat_centroid common_permit_type common_job_type  
2064      0.430405                 EW              A2  
2092      0.389282                 EW              A2  
2091     -0.255993                 EW              A2  
2128 

In [26]:
len(df_hotspot.index)

3807231

In [27]:
len(df.index)

3807231

In [28]:
df_hotspot['LATITUDE_ORIG'] = df['LATITUDE_ORIG'].copy()
df_hotspot['LONGITUDE_ORIG'] = df['LONGITUDE_ORIG'].copy()

In [29]:
import matplotlib.pyplot as plt

# Assign a color to each cluster (reusing up to 20 colors, noise in gray)
unique_clusters = df_hotspot['cluster'].unique()
num_clusters = len(unique_clusters)
colors = plt.get_cmap('tab20', num_clusters)
color_dict = {cl: colors(i % 20) for i, cl in enumerate(unique_clusters)}
color_dict[-1] = '#888888'  # gray for noise

plt.figure(figsize=(10, 10))
for cl in unique_clusters:
    subset = df_hotspot[df_hotspot['cluster'] == cl]
    plt.scatter(
        subset['LONGITUDE_ORIG'],
        subset['LATITUDE_ORIG'],
        s=5,
        color=color_dict.get(cl, '#000000'),
        label=f'Cluster {cl}' if cl != -1 else 'Noise',
        alpha=0.6
    )

plt.xlabel('Longitude')
plt.ylabel('Latitude')
plt.title('NYC Urban Development Hotspot Clusters')
plt.legend(markerscale=3, fontsize='small', bbox_to_anchor=(1.05, 1), loc='upper left', borderaxespad=0.)
plt.tight_layout()
plt.savefig('data/post_clustering/nyc_clustered_points.png', dpi=150)
plt.close()

C:\Users\nandi\AppData\Local\Temp\ipykernel_21864\1317414282.py:26: UserWarning: Tight layout not applied. The bottom and top margins cannot be made large enough to accommodate all Axes decorations.
  plt.tight_layout()


In [42]:
import matplotlib.pyplot as plt

# Assign a color to each cluster (reusing up to 20 colors, noise in gray)
unique_clusters = df_hotspot['cluster'].unique()
num_clusters = len(unique_clusters)
colors = plt.get_cmap('tab20', num_clusters)
color_dict = {cl: colors(i % 20) for i, cl in enumerate(unique_clusters)}
color_dict[-1] = '#888888'  # gray for noise

plt.figure(figsize=(10, 10))
for cl in unique_clusters:
    subset = df_hotspot[df_hotspot['cluster'] == cl]
    plt.scatter(
        subset['LONGITUDE_ORIG'],
        subset['LATITUDE_ORIG'],
        s=5,
        color=color_dict.get(cl, '#000000'),
        label=f'Cluster {cl}' if cl != -1 else 'Noise',
        alpha=0.6
    )

plt.xlabel('Longitude')
plt.ylabel('Latitude')
plt.title('NYC Urban Development Hotspot Clusters')
plt.legend(markerscale=3, fontsize='small', bbox_to_anchor=(1.05, 1), loc='upper left', borderaxespad=0.)
plt.tight_layout()
plt.savefig('data/post_clustering/nyc_clustered_points.png', dpi=150)
plt.close()


C:\Users\nandi\AppData\Local\Temp\ipykernel_33276\394161513.py:26: UserWarning: Tight layout not applied. The bottom and top margins cannot be made large enough to accommodate all Axes decorations.
  plt.tight_layout()


In [36]:
import matplotlib.pyplot as plt
from collections import Counter

plt.figure(figsize=(12, 12))

# Count cluster sizes and select most frequent clusters for the legend
cluster_sizes = Counter(df_hotspot['cluster'])
# Adjust n_legend_clusters for your case (e.g., 10 most common, plus noise if present)
n_legend_clusters = 10
most_common_clusters = [cl for cl, _ in cluster_sizes.most_common(n_legend_clusters)]
special_labels = {-1: 'Noise'}  # Label for noise

# Define cmap for coloring
unique_clusters = df_hotspot['cluster'].unique()
num_clusters = len(unique_clusters)
cmap = plt.get_cmap('tab20', num_clusters)

for idx, cl in enumerate(sorted(df_hotspot['cluster'].unique())):
    subset = df_hotspot[df_hotspot['cluster'] == cl]
    color = cmap(idx % cmap.N) if cl != -1 else '#888888'
    label = (
        special_labels[cl]
        if cl in special_labels
        else (f'Cluster {cl}' if cl in most_common_clusters else None)
    )
    plt.scatter(
        subset['LONGITUDE_ORIG'], subset['LATITUDE_ORIG'],
        s=5, color=color,
        label=label,  # Label only for legend clusters
        alpha=0.65
    )

plt.xlabel('Longitude')
plt.ylabel('Latitude')
plt.title('NYC Urban Development Hotspot Clusters')

# Create the legend outside the plot
handles, labels = plt.gca().get_legend_handles_labels()
by_label = dict(zip(labels, handles))
plt.legend(
    by_label.values(), by_label.keys(),
    title='Clusters',
    title_fontsize='small',
    fontsize='small',
    markerscale=2,
    loc='center left',
    bbox_to_anchor=(1.04, 0.5),
    borderaxespad=1,
    frameon=True
)

plt.tight_layout(rect=[0, 0, 0.85, 1])  # leave space for legend at right
plt.savefig('data/post_clustering/nyc_clustered_points_multicolor.png', dpi=150, bbox_inches='tight')
plt.close()


In [31]:
from sklearn.metrics import silhouette_score, calinski_harabasz_score, davies_bouldin_score

# Select only points assigned to a non-noise cluster if using DBSCAN
non_noise = df_hotspot['cluster'] != -1
X_eval = df_hotspot.loc[non_noise, features_for_clustering]  # use your clustering features

labels = df_hotspot.loc[non_noise, 'cluster']


# Subsample for scoring (e.g., 20,000 non-noise points)
sample_size = 50000
if len(X_eval) > sample_size:
    sample_idx = np.random.choice(X_eval.index, sample_size, replace=False)
    X_eval_sample = X_eval.loc[sample_idx]
    labels_sample = labels.loc[sample_idx]
else:
    X_eval_sample = X_eval
    labels_sample = labels

# Only compute if there is more than 1 cluster
if len(set(labels_sample)) > 1:
    sil_score = silhouette_score(X_eval_sample, labels_sample)
    ch_score = calinski_harabasz_score(X_eval_sample, labels_sample)
    db_score = davies_bouldin_score(X_eval_sample, labels_sample)
    print(f"Silhouette Score: {sil_score:.3f}")
    print(f"Calinski-Harabasz Index: {ch_score:.3f}")
    print(f"Davies-Bouldin Index: {db_score:.3f}")
else:
    print("Not enough clusters for quality scoring.")

# Cluster counts
from collections import Counter
cluster_counts = Counter(df_hotspot['cluster'])
noise_pct = 100.0 * cluster_counts[-1] / len(df_hotspot)
print(f"Cluster counts: {cluster_counts}")
print(f"Noise points: {cluster_counts[-1]} ({noise_pct:.1f}%)")

Silhouette Score: 0.500
Calinski-Harabasz Index: 34507.442
Davies-Bouldin Index: 0.441
Cluster counts: Counter({-1: 3198489, 121: 1931, 837: 1853, 82: 1823, 222: 1823, 190: 1800, 109: 1797, 0: 1790, 1488: 1786, 1616: 1748, 231: 1742, 1591: 1722, 1212: 1714, 142: 1712, 129: 1711, 170: 1676, 186: 1674, 163: 1672, 999: 1660, 115: 1656, 1845: 1654, 135: 1650, 1107: 1638, 1447: 1637, 933: 1634, 1330: 1634, 914: 1628, 1077: 1623, 1308: 1614, 56: 1608, 68: 1604, 555: 1600, 71: 1599, 128: 1598, 191: 1598, 90: 1596, 100: 1596, 103: 1593, 1366: 1592, 110: 1586, 270: 1580, 1106: 1580, 41: 1576, 40: 1572, 70: 1569, 125: 1554, 37: 1547, 36: 1546, 1879: 1543, 1202: 1542, 166: 1527, 193: 1522, 266: 1507, 676: 1501, 278: 1501, 188: 1501, 229: 1499, 1312: 1485, 683: 1480, 46: 1478, 26: 1474, 201: 1470, 1153: 1469, 451: 1467, 148: 1461, 15: 1456, 133: 1451, 1087: 1451, 210: 1448, 81: 1443, 880: 1442, 1125: 1442, 1478: 1436, 45: 1434, 5: 1431, 1190: 1430, 98: 1425, 16: 1418, 264: 1415, 291: 1415, 1108: 1

In [ ]:
print("\n" + "="*70)
print("CLUSTER INTERPRETATION")
print("="*70)

from collections import Counter
import numpy as np

plt.figure(figsize=(12, 12))

# Count cluster sizes
cluster_sizes = Counter(df_hotspot['cluster'])

# Top 10 non-noise clusters by size
non_noise = [(cl, n) for cl, n in cluster_sizes.items() if cl != -1]
non_noise_sorted = sorted(non_noise, key=lambda t: t[1], reverse=True)
top_clusters = [cl for cl, _ in non_noise_sorted[:10]]

# Borough estimation from Council District
# Manhattan: 1-10, Bronx: 11-18, Brooklyn: 33-48, Queens: 19-32, Staten Island: 49-51
def estimate_borough(council_district):
    try:
        cd = int(council_district)
        if 1 <= cd <= 10:
            return 'Manhattan'
        elif 11 <= cd <= 18 or cd == 8:
            return 'Bronx'
        elif 19 <= cd <= 32:
            return 'Queens'
        elif 33 <= cd <= 48:
            return 'Brooklyn'
        elif 49 <= cd <= 51:
            return 'Staten Island'
        else:
            return 'Unknown'
    except:
        return 'Unknown'

# One-hot column definitions
permit_type_cols = [c for c in df_hotspot.columns if c.startswith('Permit Type_')]
job_type_cols = [c for c in df_hotspot.columns if c.startswith('Job Type_')]
permit_status_cols = [c for c in df_hotspot.columns if c.startswith('Permit Status_')]

# Permit Type full names
permit_type_names = {
    'DM': 'Demolition',
    'EQ': 'Equipment',
    'EW': 'Equipment Work',
    'FO': 'Foundation',
    'NB': 'New Building',
    'PL': 'Plumbing',
    'SG': 'Sign'
}

# Job Type full names
job_type_names = {
    'A1': 'Alteration Type 1',
    'A2': 'Alteration Type 2',
    'A3': 'Alteration Type 3',
    'DM': 'Demolition',
    'NB': 'New Building',
    'SG': 'Sign'
}

# Store results for table
interpretation_results = []

for cluster_id in sorted(df_hotspot['cluster'].unique()):
# for cluster_id in top_clusters:
    cluster_data = df_hotspot[df_hotspot['cluster'] == cluster_id]
    n_samples = len(cluster_data)
    pct = n_samples / len(df_hotspot) * 100
    
    print(f"\n{'='*50}")
    print(f"CLUSTER {cluster_id}: {n_samples:,} samples ({pct:.1f}%)")
    print(f"{'='*50}")
    
    # ----- 1. Top Permit Type -----
    if permit_type_cols:
        permit_sums = cluster_data[permit_type_cols].sum()
        top_permit_col = permit_sums.idxmax()
        top_permit_code = top_permit_col.replace('Permit Type_', '')
        top_permit_name = permit_type_names.get(top_permit_code, top_permit_code)
        top_permit_pct = permit_sums[top_permit_col] / n_samples * 100
        print(f"  Top Permit Type: {top_permit_name} ({top_permit_code}) - {top_permit_pct:.1f}%")
    else:
        top_permit_name = 'N/A'
    
    # ----- 2. Top Job Type -----
    if job_type_cols:
        job_sums = cluster_data[job_type_cols].sum()
        top_job_col = job_sums.idxmax()
        top_job_code = top_job_col.replace('Job Type_', '')
        top_job_name = job_type_names.get(top_job_code, top_job_code)
        top_job_pct = job_sums[top_job_col] / n_samples * 100
        print(f"  Top Job Type: {top_job_name} ({top_job_code}) - {top_job_pct:.1f}%")
    else:
        top_job_name = 'N/A'
    
    # ----- 3. Residential Ratio -----
    if 'Residential_YES' in cluster_data.columns:
        residential_pct = cluster_data['Residential_YES'].mean() * 100
        residential_label = 'Residential' if residential_pct > 50 else 'Commercial/Mixed'
        print(f"  Residential: {residential_pct:.1f}% → {residential_label}")
    else:
        residential_pct = 0
        residential_label = 'Unknown'
    
    # ----- 4. Permit Status -----
    if permit_status_cols:
        status_sums = cluster_data[permit_status_cols].sum()
        top_status_col = status_sums.idxmax()
        top_status = top_status_col.replace('Permit Status_', '')
        top_status_pct = status_sums[top_status_col] / n_samples * 100
        print(f"  Top Permit Status: {top_status} - {top_status_pct:.1f}%")
    else:
        top_status = 'N/A'
    
    # ----- 5. Temporal Features -----
    if 'Filing_Year' in cluster_data.columns:
        avg_year = cluster_data['Filing_Year'].mean()
        print(f"  Avg Filing Year: {avg_year:.1f}")
    else:
        avg_year = 0
    
    if 'Permit_Age_Days' in cluster_data.columns:
        avg_age = cluster_data['Permit_Age_Days'].mean()
        print(f"  Avg Permit Processing Time: {avg_age:.1f} days")
    else:
        avg_age = 0
    
    # ----- 6. Geographic Center -----
    if 'LATITUDE_ORIG' in cluster_data.columns and 'LONGITUDE_ORIG' in cluster_data.columns:
        lat_center = cluster_data['LATITUDE_ORIG'].mean()
        long_center = cluster_data['LONGITUDE_ORIG'].mean()
        print(f"  Geographic Center: ({lat_center:.4f}, {long_center:.4f})")
    else:
        lat_center, long_center = 0, 0
    
    # ----- 7. Estimate Borough from Council District -----
    if 'COUNCIL_DISTRICT' in cluster_data.columns:
        # Unscale if needed (assuming it was standardized)
        # For now, use mode of rounded values
        try:
            borough_counts = cluster_data['COUNCIL_DISTRICT'].apply(
                lambda x: estimate_borough(round(x * 10 + 25))  # rough unscaling
            ).value_counts()
            top_borough = borough_counts.index[0] if len(borough_counts) > 0 else 'Unknown'
        except:
            top_borough = 'Unknown'
        print(f"  Estimated Borough: {top_borough}")
    else:
        top_borough = 'Unknown'
    
    # ----- 8. Generate Cluster Label -----
    # Create a human-readable cluster description
    if n_samples < 1000:
        size_label = "Specialized/Rare"
    elif pct < 5:
        size_label = "Small"
    elif pct < 20:
        size_label = "Medium"
    else:
        size_label = "Large"
    
    cluster_label = f"{size_label} {residential_label} - {top_permit_name}"
    print(f"\n  → CLUSTER LABEL: {cluster_label}")
    
    # Store for summary table
    interpretation_results.append({
        'Cluster': cluster_id,
        'Size': n_samples,
        'Pct': f"{pct:.1f}%",
        'Top_Permit': top_permit_name,
        'Top_Job': top_job_name,
        'Residential%': f"{residential_pct:.1f}%",
        'Avg_Year': f"{avg_year:.0f}" if avg_year > 0 else 'N/A',
        'Processing_Days': f"{avg_age:.0f}" if avg_age > 0 else 'N/A',
        'Label': cluster_label
    })

# ============================================================
# SUMMARY TABLE
# ============================================================
print("\n" + "="*70)
print("CLUSTER INTERPRETATION SUMMARY TABLE")
print("="*70)

summary_df = pd.DataFrame(interpretation_results)
print(summary_df.to_string(index=False))

# Save to CSV
summary_df.to_csv('data/post_clustering/cluster_interpretation_dbscan.csv', index=False)
print("\n✓ Saved: data/post_clustering/cluster_interpretation_dbscan.csv")

# ============================================================
# MARKDOWN TABLE FOR PRESENTATION
# ============================================================
print("\n" + "="*70)
print("MARKDOWN TABLE (Copy for Presentation)")
print("="*70)

print("\n| Cluster | Size | % | Top Permit | Top Job | Residential | Label |")
print("|---------|------|---|------------|---------|-------------|-------|")
for r in interpretation_results:
    print(f"| {r['Cluster']} | {r['Size']:,} | {r['Pct']} | {r['Top_Permit']} | {r['Top_Job']} | {r['Residential%']} | {r['Label']} |")

print("\n" + "="*70)
print("CLUSTER INTERPRETATION COMPLETE")
print("="*70)


CLUSTER INTERPRETATION

CLUSTER 121: 1,931 samples (0.1%)
  Top Permit Type: Equipment Work (EW) - 100.0%
  Top Job Type: Alteration Type 2 (A2) - 100.0%
  Residential: 0.0% → Commercial/Mixed
  Top Permit Status: ISSUED - 100.0%
  Avg Filing Year: 2018.0
  Avg Permit Processing Time: 0.0 days
  Geographic Center: (40.7455, -73.9882)
  Estimated Borough: Bronx

  → CLUSTER LABEL: Small Commercial/Mixed - Equipment Work

CLUSTER 837: 1,853 samples (0.0%)
  Top Permit Type: Equipment Work (EW) - 100.0%
  Top Job Type: Alteration Type 2 (A2) - 100.0%
  Residential: 0.0% → Commercial/Mixed
  Top Permit Status: ISSUED - 100.0%
  Avg Filing Year: 2017.0
  Avg Permit Processing Time: 0.0 days
  Geographic Center: (40.7449, -73.9877)
  Estimated Borough: Bronx

  → CLUSTER LABEL: Small Commercial/Mixed - Equipment Work

CLUSTER 82: 1,823 samples (0.0%)
  Top Permit Type: Equipment Work (EW) - 100.0%
  Top Job Type: Alteration Type 2 (A2) - 100.0%
  Residential: 0.0% → Commercial/Mixed
  Top P

<Figure size 1200x1200 with 0 Axes>

In [34]:
df_hotspot.columns

Index(['LATITUDE', 'LONGITUDE', 'COUNCIL_DISTRICT', 'CENSUS_TRACT',
       'Bldg Type', 'Filing_Year', 'Filing_Month', 'Filing_Quarter',
       'Permit_Age_Days', 'Permit Type_DM', 'Permit Type_EQ', 'Permit Type_EW',
       'Permit Type_FO', 'Permit Type_NB', 'Permit Type_PL', 'Permit Type_SG',
       'Job Type_A2', 'Job Type_A3', 'Job Type_DM', 'Job Type_NB',
       'Job Type_SG', 'Permit Status_ISSUED', 'Permit Status_RE-ISSUED',
       'Residential_YES', 'cluster', 'is_hotspot', 'LATITUDE_ORIG',
       'LONGITUDE_ORIG'],
      dtype='object')

In [ ]:
import matplotlib.pyplot as plt
from collections import Counter
import numpy as np

plt.figure(figsize=(12, 12))

# Count cluster sizes
cluster_sizes = Counter(df_hotspot['cluster'])

# Top 10 non-noise clusters by size
non_noise = [(cl, n) for cl, n in cluster_sizes.items() if cl != -1]
non_noise_sorted = sorted(non_noise, key=lambda t: t[1], reverse=True)
top_clusters = [cl for cl, _ in non_noise_sorted[:10]]

print("Top 10 DBSCAN clusters:", [(cl, cluster_sizes[cl]) for cl in top_clusters])

# Colors: use a nice continuous colormap for top clusters
cmap = plt.get_cmap('tab10')          # or 'tab20', 'Set1', etc.
cluster_to_color = {
    cl: cmap(i % cmap.N) for i, cl in enumerate(top_clusters)
}

# Plot all noise + non-top clusters in light gray first (background)
is_noise_or_other = ~df_hotspot['cluster'].isin(top_clusters)
bg = df_hotspot[is_noise_or_other]
plt.scatter(
    bg['LONGITUDE_ORIG'], bg['LATITUDE_ORIG'],
    s=4,
    color='#d0d0d0',       # light gray
    alpha=0.35,
    label='Noise / other clusters'
)

# Now plot only top 10 clusters in color
for i, cl in enumerate(top_clusters):
    subset = df_hotspot[df_hotspot['cluster'] == cl]
    color = cluster_to_color[cl]
    label = f'Cluster {cl} (n={cluster_sizes[cl]:,})'
    plt.scatter(
        subset['LONGITUDE_ORIG'], subset['LATITUDE_ORIG'],
        s=6,
        color=color,
        alpha=0.9,
        label=label
    )

plt.xlabel('Longitude')
plt.ylabel('Latitude')
plt.title('NYC Urban Development Hotspot Clusters (Top 10 DBSCAN clusters)')

# Legend outside plot, deduplicated
handles, labels = plt.gca().get_legend_handles_labels()
by_label = dict(zip(labels, handles))
plt.legend(
    by_label.values(), by_label.keys(),
    title='Clusters',
    title_fontsize='small',
    fontsize='small',
    markerscale=2,
    loc='center left',
    bbox_to_anchor=(1.04, 0.5),
    borderaxespad=1,
    frameon=True
)

plt.tight_layout(rect=[0, 0, 0.85, 1])
plt.savefig(
    'data/post_clustering/nyc_dbscan_top10_clusters.png',
    dpi=150,
    bbox_inches='tight'
)
plt.close()


Top 10 DBSCAN clusters: [(121, 1931), (837, 1853), (82, 1823), (222, 1823), (190, 1800), (109, 1797), (0, 1790), (1488, 1786), (1616, 1748), (231, 1742)]
